# Question 3:

Baten, Tack, and Baeder (1958) compared judges' scores on fish prepared by three methods. Twelve fish were cooked by each method, and several judges tasted fish samples and rated each on four variables:  
  
$y_1 =$ aroma,  
$y_2 =$ flavor,  
$y_3 =$ texture,  
$y_4 =$ moisture  
  
Each entry is an aveage score for the judges on that fish.

In [2]:
data <- read.table(file = "/Users/alexwiigs/Documents/Code/STAT 553 Assigmnents/Assignment 3/T6_17_FISH.DAT")
colnames(data) = c("Method", "Aroma", "flavor", "Texture", "Moisture")
method1 <- as.matrix(data[c(data[,1] == 1), 2:5])
method2 <- as.matrix(data[c(data[,1] == 2), 2:5])
method3 <- as.matrix(data[c(data[,1] == 3), 2:5])

### a) Compare the three methods using MANOVA tests (Wilks' and Roy's tests).

First, calculate the necessesary calculations:

In [3]:
# Preliminary Calculations for H
method1. <- matrix(nrow =4, ncol =1)
for (i in 1:4) { method1.[i,] = sum(method1[,i]) }
method2. <- matrix(nrow =4, ncol =1)
for (i in 1:4) { method2.[i,] = sum(method2[,i]) }
method3. <- matrix(nrow =4, ncol =1)
for (i in 1:4) { method3.[i,] = sum(method3[,i]) }
method.. <- matrix(nrow =4, ncol =1)
for (i in 1:4) { method..[i,] = sum(method1[,i]) + sum(method2[,i]) + sum(method3[,i]) }
n = 12
k = 3

In [4]:
method1.

64.6
68.8
65.3
71.8


In [20]:
# H matrix Calculations
H <- (1/n) * ( method1. %*% t(method1.) + method2. %*% t(method2.) + method3. %*% t(method3.) ) - ( 1 / (k * n) ) * ( method.. %*% t(method..) )
round(H,4)

1.0506,2.1733,-1.3756,-0.7603
2.1733,4.8800,-2.3733,-1.2567
-1.3756,-2.3733,2.3822,1.3844
-0.7603,-1.2567,1.3844,0.8106


In [21]:
# E matrix Calculations
E <- matrix(nrow = 4, ncol = 4)
E[,] = 0
for (i in 1:12) {E = E + method1[i,] %*% t(method1[i,]) + method2[i,] %*% t(method2[i,]) + method3[i,] %*% t(method3[i,])}
E = E - (1/n) * (method1. %*% t(method1.) + method2. %*% t(method2.) + method3. %*% t(method3.))
round(E,4)

Aroma,flavor,Texture,Moisture
13.4083,7.7233,8.6750,5.8642
7.7233,8.4800,7.5267,6.2133
8.6750,7.5267,11.6075,7.0375
5.8642,6.2133,7.0375,10.5658


**Wilk's Test Statistic**

For the scores on fish prepared in three different methods, we have have:  
$H_0: \mu_1 = \mu_2 = \mu_3$,  
$H_1:$ Some $\mu_i$ does not equal the others.  


The test statistic to be calculated will be: $$\Lambda = \frac{|E|}{|E+H|} \sim \Lambda_{p = 4, v_H = 2, v_E = 33}$$  
Calculating our test statistic we have:


In [22]:
# Wilk's Test Statistic
Lambda_obs <- det(E) / det(E + H )
sprintf("The calculated Wilks Test Statistic is %f.", Lambda_obs)

[1] "The calculated Wilks Test Statistic is 0.224487."

Table A.9 in Methods of Multivariate Analysis provides the critical value $$\Lambda_{\alpha = 0.05, p=4, v_H=2,v_E = 33}  \approx .580$$  
Our observed test statistic is lower than the critcal value, therefore we reject the null hypothesis that there is no difference in the scores based on the different cooking methods.

Since $v_H = 2$, we can calculate the p-value by transferring into an F-statistic, like so:

In [23]:
pf((1-sqrt(Lambda_obs)) / sqrt(Lambda_obs) * (33 - 4 + 1)/4, 8, 2 * (33-4 +1), lower.tail = FALSE)

[1] 1.608771e-07

Again, we reject the null hypoethesis.

**Roy's Test Statistic**

For the scores on fish prepared in three different methods, we have have:  
$H_0: \mu_1 = \mu_2 = \mu_3$,  
$H_1:$ Some $\mu_i$ does not equal the others.  

The test statistic to be calculated will be: $$\theta = \frac{\lambda_1}{1 + \lambda_1} \sim \theta_{s=2, m=1, N=14}$$  
where $\lambda_1$ is the greatest eigenvalue of $E^{-1} H$.  

Calculating our test statistic we have:

In [24]:
# Roy's Test Statistic
eigenvalues <- eigen( solve(E) %*% H)$values
theta <- max(eigenvalues) / (1 + max(eigenvalues))
sprintf("The calculated Roys Test Statistic is %f.", theta)



[1] "The calculated Roys Test Statistic is 0.746930."

Table A.10 in Methods of Multivariate Analysis provides the critical value
$$\theta_{\alpha = 0.05, s=2,m=1,N=14} \approx .348

Our observed test statistic is greater than the critical value, so we reject the null hypothesis. 

We can estimate the upper bound F statistic to estimate the p-value like so: (using 6.21 in Methods of Multivariate Analysis)

In [25]:
pf((33-4-1)* max(eigenvalues) / 4, 4, 33-4-1, lower.tail = FALSE)

[1] 5.063393e-08

We reject the Null hypothesis.

b) What assumptions do you need to perform the tests in part a)?

For both Wilks' Test and Roy's Test we assume:  
1. The data within each group has a multivariate normal distribution.
2. The populations have the same covariance matrices (homogeneity).
3. The observations are independent.
4. The observations are drawn randomly from the population.

c) If we multiply the scores for $y_1$ by 2 and multiply the scores for $y_2$ by 3, then we perform the MANOVA tests to compare the three methods. Are the observed test statistics the same as the in part a)? Can you derive a general result?

In [26]:
# redefine variables
method1alt <- cbind(2*method1[,1], 3*method1[,2], method1[,3], method1[,4])
method2alt <- cbind(2*method2[,1], 3*method2[,2], method2[,3], method2[,4])
method3alt <- cbind(2*method3[,1], 3*method3[,2], method3[,3], method3[,4])

method1alt. <- matrix(nrow =4, ncol =1)
for (i in 1:4) { method1alt.[i,] = sum(method1alt[,i]) }
method2alt. <- matrix(nrow =4, ncol =1)
for (i in 1:4) { method2alt.[i,] = sum(method2alt[,i]) }
method3alt. <- matrix(nrow =4, ncol =1)
for (i in 1:4) { method3alt.[i,] = sum(method3alt[,i]) }
methodalt.. <- matrix(nrow =4, ncol =1)
for (i in 1:4) { methodalt..[i,] = sum(method1alt[,i]) + sum(method2alt[,i]) + sum(method3alt[,i]) }
n = 12
k = 3

In [27]:
# alternative H
Halt <- (1/n) * ( method1alt. %*% t(method1alt.) + method2alt. %*% t(method2alt.) + method3alt. %*% t(method3alt.) ) - ( 1 / (k * n) ) * ( methodalt.. %*% t(methodalt..) )

In [28]:
# alternative E
Ealt <- matrix(nrow = 4, ncol = 4)
Ealt[,] = 0
for (i in 1:12) {Ealt = Ealt + method1alt[i,] %*% t(method1alt[i,]) + method2alt[i,] %*% t(method2alt[i,]) + method3alt[i,] %*% t(method3alt[i,])}
Ealt = Ealt - (1/n) * (method1alt. %*% t(method1alt.) + method2alt. %*% t(method2alt.) + method3alt. %*% t(method3alt.))

In [29]:
# Alternative Wilks' Statistic.
Lambda_alt <- det(Ealt) / det(Ealt + Halt )
sprintf("The calculated Wilks Test Statistic is %f.", Lambda_alt)

[1] "The calculated Wilks Test Statistic is 0.224487."

In [30]:
# Roy's Test Statistic
eigenvaluesalt <- eigen( solve(Ealt) %*% Halt)$values
thetaalt <- max(eigenvaluesalt) / (1 + max(eigenvaluesalt))
sprintf("The calculated Roys Test Statistic is %f.", thetaalt)

[1] "The calculated Roys Test Statistic is 0.746930."

Both Test Statistics are unchanged given the the transformation of the two variables. We conclude that scaling the variables does not change the test statstics for the MANOVA results. 